# Crear, Modificar y Eliminar Posts

En esta parte vamos a darle al usuario la posibilidad de crear, modificar o eliminar posts.

Para esto vamos a usar **`class-based views`**, esto no es más que una vista que en lugar de definirla como función la definimos como una clase, esto nos permite agregar mas funcionalidad a la vista.

**`django`** tiene un modulo llamado **`views.generic`** donde estan definidas estas clases, éstas son conocidas como **`generic views`**.

En la **`app`** de **`blog`** vamos a **`views.py`** y agregamos:

```python
from django.views.generic import ListView

class PostListView(ListView):
    model = Post
    template_name = 'blog/home.html'
    context_object_name = 'posts'
    ordering = ['-date_posted']
    
```

- Creamos una nueva clase llamada **`PostListView`** que hereda de la clase **`ListView`**.


- Esta nueva clase necesita un atributo llamado **`model`**, esta variable será el nombre de la clase/tabla a la que hacemos referencia. En este caso la clase/tabla se llama **`Post`**.


- Por defecto esta clase buscará el **`template`** con nombre **`blog/post_list.html`** (**`<app>/<model>_<viewtype>.html`**), por lo que si queremos que utilice el **`template`** que ya teniamos, debemos modificar el atributo **`template_name`**.


- Al igual que en la función **`home`**, tenemos que decirle a la clase como se llamará el objeto sobre el que va a iterar, en la función **`home`** lo llamamos **`posts`**, por lo que debemos modificar el nombre del atributo **`context_object_name`** a **`posts`**. Por defecto este objeto se llama **`object`**.


- **`ordering`** es un atributo que ordena la lista que vamos a iterar, en este ejemplo queremos ordenar por **`date_posted`** pero que los más recientes aparezcan de primero, para eso agregamos un **`-`** delante de la atributo.

**Esta vista va a sustituir la vista de `home`**.

Ahora tenemos que agregar esta vista a las url de la app. Vamos a **`urls.py`** y agregamos:

```python
from .views import PostListView
```
Y en la lista **`urlspatterns`** modificamos el url de la vista **`home`**:

```python
path('', PostListView.as_view(), name='blog-home')
```

El método **`.as_view()`** "transforma" la clase a una vista, lo que permite utilizar la clase como si fuese la función **`home`**.

**`Ejecutamos el servidor y vemos los resultados.`**

------


Ahora vamos a crear una vista para **"ver"** los posts de los usuarios. Para esto en **`views.py`** agregamos:

```python
from django.views.generic import DetailView

class PostDetailView(DetailView):
    model = Post
```
- Ésta clase intentará buscar el template **`blog/post_detail.html`** pero este archivo no existe, lo vamos a crear luego de agregar un url a esta vista.

Ahora vamos a crear el url para esta vista. Vamos a **`urls.py`**:

```python
from .views import PostDetailView
```
Y en la lista **`urlspatterns`** agregamos:

```python
path('post/<int:pk>/', PostDetailView.as_view(), name='post-detail')
```

- **`'post/<int:pk>/'`** es un url dinámico, que toma como "parametro" la **`primary key`** (pk) del posts, por lo que va a cambiar dependiendo del post en el que estemos.

Ahora vamos a crear un **`template`** para esta vista, vamos al directorio de **`templates/blog`** y creamos un fichero nuevo llamado **`post_detail.html`**:

```html
{% extends "blog/base.html" %}
{% block contenido %}
  <article class="media content-section">
    <img class="rounded-circle article-img" src="{{ object.author.profile.image.url }}">
    <div class="media-body">
      <div class="article-metadata">
        <a class="mr-2" href="#">{{ object.author }}</a>
        <small class="text-muted">{{ object.date_posted|date:"F d, Y" }}</small>
      </div>
      <h2 class="article-title">{{ object.title }}</h2>
      <p class="article-content">{{ object.content }}</p>
    </div>
  </article>
{% endblock contenido %}
```

Ahora podemos ver cada post individualmente usando su propio url, pero aún debemos modificar del **`template`** **`home`** para poder acceder desde esta vista.

En el **`template`** **`home`** modificamos la linea:
```html
<h2><a class="article-title" href="#">{{ post.title }}</a></h2>
```

*Por:*

```html
<h2><a class="article-title" href="{% url 'post-detail' post.id %}">{{ post.title }}</a></h2>
```

**`Ejecutamos el servidor y vemos los resultados.`**

------


## Crear posts

Al igual que la parte anterior, vamos a hacer uso de las **`generic views`**.

En **`views.py`** agregamos:

```python
from django.views.generic import CreateView
from django.contrib.auth.mixins import LoginRequiredMixin

class PostCreateView(LoginRequiredMixin, CreateView):
    model = Post
    fields = ['title', 'content']
    # Si quisieramos todos los campos usariamos '__all__'
    
    def form_valid(self, form):
        form.instance.author = self.request.user
        return super().form_valid(form)
```

- La función **`form_valid()`** asigna a la creación del post el usuario que esta "loggeado", sin este parametro no podriamos crear un post, ya que de la clase/tabla **`Post`** el usuario no puede ser nulo.

- La clase **`LoginRequiredMixin`** se encarga de comprobar que haya un usuario "loggeado" en el sistema. De lo contrario nos retornará a la página de "log-in".

Ahora vamos a crear el url en **`urls.py`**:

```python
from .views import PostCreateView
```
Y en la lista **`urlspatterns`** agregamos:
```python
path('post/new/', PostCreateView.as_view(), name='post-create')
```

Y para el template **`post_form.html`**:

```html
{% extends "blog/base.html" %}
{% load crispy_forms_tags %}
{% block contenido %}
    <div class="content-section">
        <form method="POST">
            {% csrf_token %}
            <fieldset class="form-group">
                <legend class="border-bottom mb-4">Blog Post</legend>
                {{ form|crispy }}
            </fieldset>
            <div class="form-group">
                <button class="btn btn-outline-info" type="submit">Post</button>
            </div>
        </form>
    </div>
{% endblock contenido %}
```

El nombre de este **`template`** no sigue el mismo patrón que los otros, esto es algo de **`django`** y la idea es seguir estas convenciones.

Si ejecutamos el servidor e intentamos crear un nuevo post **`django`** creará el post, pero nos retornará a una página que no existe y nos dirá que es porque no hemos asignado un url para que sea redirigido después de crear un post. Para solucionar este problema debemos modificar la clase **`Post`** del fichero **`models.py`**:

```python
from django.urls import reverse
```

*Agregamos a la clase el siguiente método:*

```python
def get_absolute_url(self):
    return reverse('post-detail', kwargs={'pk' : self.pk})
```

**`Ejecutamos el servidor y vemos los resultados. Para este momento somos capaces de crear posts con nuestro usuario.`**

------

## Modificar un post

*Vamos a repetir lo mismo de la sección anterior.*

En **`views.py`** agregamos:
```python
from django.views.generic import UpdateView
from django.contrib.auth.mixins import UserPassesTestMixin

class PostUpdateView(LoginRequiredMixin, UserPassesTestMixin, UpdateView):
    model = Post
    fields = ['title', 'content']
    
    def form_valid(self, form):
        form.instance.author = self.request.user
        return super().form_valid(form)
    
    def test_func(self):
        post = self.get_object()
        if self.request.user == post.author:
            return True
        return False
```

- La clase **`UserPassesTestMixin`** verifica que el usuario que esta "loggeado" es el mismo que el autor del post. Esto es para que solamente el autor del post pueda modificarlo y no cualquier otro usuario.

Vamos a **`urls.py`**:

```python
from .views import PostUpdateView
```
Y en la lista **`urlspatterns`** agregamos:
```python
path('post/<int:pk>/update/', PostUpdateView.as_view(), name='post-update')
```

-----

## Eliminar posts

*Vamos a repetir lo mismo de la sección anterior.*

En **`views.py`** agregamos:

```python
from django.views.generic import DeleteView

class PostDeleteView(LoginRequiredMixin, UserPassesTestMixin, UpdateView):
    model = Post
    success_url = '/'
    
    def test_func(self):
    post = self.get_object()
    if self.request.user == post.author:
        return True
    return False
```
- El atributo **`success_url`** es donde la vista nos va a retornar luego de eliminar un post. En este ejemplo nos retornará a la pagina principal.

Vamos a **`urls.py`**:

```python
from .views import PostDeleteView
```
Y en la lista **`urlspatterns`** agregamos:
```python
path('post/<int:pk>/delete/', PostDeleteView.as_view(), name='post-delete')
```

Para el **`template`** de esta vista vamos a crear **`post_confirm_delete.html`** (nombre por defecto en **`django`**):

```html
{% extends "blog/base.html" %}
{% block contenido %}
    <div class="content-section">
        <form method="POST">
            {% csrf_token %}
            <fieldset class="form-group">
                <legend class="border-bottom mb-4">Delete Post</legend>
                <h2>Are you sure you want to delete the post "{{ object.title }}"</h2>
            </fieldset>
            <div class="form-group">
                <button class="btn btn-outline-danger" type="submit">Yes, Delete</button>
                <a class="btn btn-outline-secondary" href="{% url 'post-detail' object.id %}">Cancel</a>
            </div>
        </form>
    </div>
{% endblock contenido %}
```

**`Ejecutamos el servidor y vemos los resultados.`**

-----

## Agregar "New Post" a la barra de navegación

En el **`template`** **`base.html`** vamos a escribir en la barra de navegación dentro del condicional **`if`**:

```html
<a class="nav-item nav-link" href="{% url 'post-create' %}">New Post</a>
```

Ahora podemos crear un post desde la barra de navegación.


-----

## Agregar "Update Post" y "Delete Post" a la vista "post_detail.html":

En el **`template`** **`post_detail.html`** vamos a escribir después de la fecha lo siguiente:

```html
{% if object.author == user %}
  <div>
    <a class="btn btn-secondary btn-sm mt-1 mb-1" href="{% url 'post-update' object.id %}">Update</a>
    <a class="btn btn-danger btn-sm mt-1 mb-1" href="{% url 'post-delete' object.id %}">Delete</a>
  </div>
{% endif %}
```
Ahora podemos modificar los posts o eliminarlos.

**`Ejecutamos el servidor y vemos los resultados.`**